In [1]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [ ]:


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            
            driver.quit() # Close the driver
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('chandigarh_bus_details.csv', index=False)





Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/dinanagar-punjab-to-chandigarh: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6DC0D6CF5+28821]
	(No symbol) [0x00007FF6DC043880]
	(No symbol) [0x00007FF6DBEE578A]
	(No symbol) [0x00007FF6DBF391BE]
	(No symbol) [0x00007FF6DBF394AC]
	(No symbol) [0x00007FF6DBF82647]
	(No symbol) [0x00007FF6DBF5F33F]
	(No symbol) [0x00007FF6DBF7F412]
	(No symbol) [0x00007FF6DBF5F0A3]
	(No symbol) [0x00007FF6DBF2A778]
	(No symbol) [0x00007FF6DBF2B8E1]
	GetHandleVerifier [0x00007FF6DC40FCED+3408013]
	GetHandleVerifier [0x00007FF6DC42745F+3504127]
	GetHandleVerifier [0x00007FF6DC41B63D+3455453]
	GetHandleVerifier [0x00007FF6DC19BDFB+835995]
	(No symbol) [0x00007FF6DC04EB9F]
	(No symbol) [0x00007FF6DC04A854]
	(No symbol) [0x00007FF6DC04A9ED]
	(No symbol) [0x00007FF6DC03A1D9]
	BaseThreadInitThunk [0x00007FFEAC89259D+29]
	RtlUserThreadStart [0x00007FFEAD26AF38+40]

Error occurred while scraping bus details for https://www.redbu

In [3]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,HRTC - 551,A/C Executive (2+3),15:30,05h 55m,21:25,3.9,INR 400,41 Seats available
1,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,HRTC - 525,Ordinary 3+2 Non AC Seater,16:50,05h 50m,22:40,3.9,INR 290,27 Seats available
2,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,HRTC - 526,Ordinary 3+2 Non AC Seater,17:00,06h 00m,23:00,3.9,INR 275,35 Seats available
3,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,HRTC - 532,Ordinary 3+2 Non AC Seater,17:59,05h 41m,23:40,3.9,INR 290,27 Seats available
4,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,HRTC - 587,Ordinary 3+2 Non AC Seater,18:25,04h 40m,23:05,3.9,INR 290,32 Seats available
...,...,...,...,...,...,...,...,...,...,...
335,Chandigarh to Shimla,https://www.redbus.in/bus-tickets/chandigarh-t...,HRTC - 2040086,Ordinary 3+2 Non AC Seater,20:00,04h 10m,00:10,3.9,INR 242,37 Seats available
336,Chandigarh to Shimla,https://www.redbus.in/bus-tickets/chandigarh-t...,HRTC - 1729,Ordinary 3+2 Non AC Seater,20:20,05h 10m,01:30,3.9,INR 242,36 Seats available
337,Chandigarh to Shimla,https://www.redbus.in/bus-tickets/chandigarh-t...,City Land Travels,Volvo A/C Semi Sleeper (2+2),15:00,03h 30m,18:30,3.5,405,11 Seats available
338,Chandigarh to Shimla,https://www.redbus.in/bus-tickets/chandigarh-t...,City Land Travels,Volvo A/C Semi Sleeper (2+2),15:10,03h 20m,18:30,3.7,405,13 Seats available
